In [1]:
%load_ext autoreload
%autoreload 2
from yfpy.query import YahooFantasySportsQuery
import sys
sys.path.append('../src')

In [3]:
import pandas as pd
def get_players(q):
    players = q.get_league_players()
    pdf = pd.DataFrame([{
    'pos':p.display_position.split(','), 
    'name': p.name.full, 
    'player_key':p.player_key, 
    'team': p.editorial_team_abbr,
    'status': p.status 
     } for p in players])
    return pdf

def get_ts(t):
    return pd.Timestamp(t, unit='s', tz='UTC')\
        .tz_convert('US/Pacific')

def iso_get_ts(t):
    return pd.Timestamp(t).tz_convert('US/Pacific')

def get_gameweek(date):
    import requests
    games = []
    url = f'https://api-web.nhle.com/v1/schedule/{date}'
    r = requests.get(url)
    if r.status_code != 200:
        print(r.text)
    data = r.json()
    for week in data['gameWeek']:
        for game in week['games']:
            games.append({
                'gameId': game['id'],
                'home': game['homeTeam']['abbrev'],
                'away': game['awayTeam']['abbrev'],
                'ts': iso_get_ts(game['startTimeUTC']).date()
            })
    return games, data.get('nextStartDate')

def get_schedule(dates):
    games = []
    next_start = dates[0].strftime('%Y-%m-%d')
    for date in dates:
        if date >= pd.Timestamp(next_start):
            g, next_start = get_gameweek(date.strftime('%Y-%m-%d'))
            games += g
    games = [g for g in games if dates[0].date() <= g['ts'] <= dates[-1].date()]
    return games

game_id = 453

league_id = 21834
q = YahooFantasySportsQuery(
    game_id = game_id,
    league_id = league_id,
    game_code = 'nhl',
    auth_dir = '/home/jupyter/creds',
    browser_callback = False,
)


df = pd.read_csv('projections.csv')
df.columns = df.columns.str.lower()
df

,player,gp,g,a,p,+/-,pim,ppg,ppa,ppp,...,w,l,ga,gaa,sa,sv,sv%,so,otl,gx
0,A.J. Greer,74.2,6.8,8.1,15.0,-5.2,74.0,0.3,0.0,0.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aaron Ekblad,72.5,11.4,29.5,40.9,20.4,66.8,3.8,7.7,14.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Adam Beckman,49.7,1.0,6.7,7.7,-11.0,42.3,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Adam Boqvist,69.0,8.4,21.4,29.9,-15.0,11.8,0.6,7.8,9.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Adam Edstrom,48.7,8.0,0.3,8.3,6.5,8.7,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,Zachary Bolduc,41.5,7.5,7.0,14.5,0.0,8.5,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
833,Zachary Jones,68.0,3.8,13.3,17.0,-8.0,17.8,0.0,0.2,0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
834,Zack MacEwen,63.0,3.7,4.6,8.4,-13.4,86.1,0.2,0.4,0.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
835,Zack Ostapchuk,43.0,3.5,4.5,8.0,-1.0,8.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
game_id = 427

q = YahooFantasySportsQuery(
    game_id = game_id,
    league_id = league_id,
    game_code = 'nhl',
    auth_dir = '/home/jupyter/creds',
    browser_callback = False,
)

# all_matchups = []
# for i in range(1, 27):
#     all_matchups += q.get_league_matchups_by_week(i)
    
stats_map = {
    1: 'g',
    2: 'a',
    4: '+/-',
    5: 'pim',
    8: 'ppp',
    14: 'sog',
    16: 'fow',
    31: 'hit',
    32: 'blk',
    19: 'w',
    25: 'sv',
    27: 'so'
}
stats = []
for match in all_matchups:
    for mt in match.teams:
        for s in mt.team_stats.stats:
            stats.append({'id': s.stat_id, 'val': s.value})
stats = pd.DataFrame(stats)
stats['name'] = stats.id.apply(lambda x: stats_map.get(x, None))
s_exp = stats.groupby('name')['val'].mean()
s_exp

name
+/-      3.009615
a       22.153846
blk     40.389423
fow    111.913462
g       13.100962
hit     46.043269
pim     23.072115
ppp     11.168269
so       0.317308
sog    112.197115
sv     145.115385
w        3.000000
Name: val, dtype: float64

In [4]:
game_id = 453
q = YahooFantasySportsQuery(
    game_id = game_id,
    league_id = league_id,
    game_code = 'nhl',
    auth_dir = '/home/jupyter/creds',
    browser_callback = False,
)

In [157]:
def get_days_games(date, game_df, players, selected_team):
    position_lookup = players.set_index('name')['pos'].to_dict()
    position_limits = {
        'D': 4,
        'C': 2,
        'RW': 2,
        'LW': 2,
        'G': 3
    }
    todays_games = game_df[pd.to_datetime(game_df.ts) == date]
    
    todays_players = todays_games[todays_games.name.isin(selected_team)].name.tolist()

    is_included = todays_games.name.apply(lambda x: can_include_player(x, position_lookup, todays_players, position_limits, wildcard=None)).values
    is_inteam = todays_games.name.isin(selected_team).values
    todays_games = todays_games[is_included|is_inteam]
    return todays_games

def get_week_games(dates, game_df, players, selected_team):
    week_games = []
    for date in dates:
        week_games.append(get_days_games(date, game_df, players, selected_team))
    week_games = pd.concat(week_games)
    return week_games

In [318]:
from scipy import stats

def get_opponent(m):
    m_t = [t.team_key for t in m.teams]
    if own_team in m_t:
        opponent = [t for t in m_t if t != own_team][0]
    return opponent

from scipy import stats
def get_added_val(x, opp_exp, stat_cols, selected_team):
    vals = {}
    for col in stat_cols:
        vals[col] = stats.skellam.cdf(0, opp_exp[col] / (17 - len(selected_team)), x[col])[0]
    return vals

def get_added_vals(all_preds, opp_exp, selected_team):
    total_vals = all_preds.apply(lambda x: pd.Series(get_added_val(x, opp_exp, ycols, selected_team)), 1).fillna(0)
    return total_vals

def can_include_player(new_player, position_lookup, selected_players, position_limits, wildcard=None):
    from collections import defaultdict

    # Helper function to check if the current assignment of players to positions is valid
    def is_valid_assignment(assignments):
        counts = defaultdict(int)
        for player, pos in assignments.items():
            counts[pos] += 1
            if counts[pos] > position_limits.get(pos, 0):
                return False
        return True
    
    # Helper function for the backtracking algorithm
    def backtrack(assignments, players):
        if not players:
            return is_valid_assignment(assignments)
        
        current_player = players[0]
        possible_positions = position_lookup.get(current_player, [])
        
        # Include wildcard positions if the current player is the wildcard
        if current_player == wildcard:
            possible_positions = position_limits.keys()
        
        for pos in possible_positions:
            assignments[current_player] = pos
            if is_valid_assignment(assignments):
                if backtrack(assignments, players[1:]):
                    return True
            assignments.pop(current_player, None)
        return False
    
    # List of players to be considered (already selected + new player)
    all_players = selected_players + [new_player]
    
    # Initial assignments
    initial_assignments = {}
    
    # Run the backtracking algorithm
    return backtrack(initial_assignments, all_players)

In [105]:
players = get_players(q)
players = players[~((players.name.isin(['Sebastian Aho','Elias Pettersson']))&(players.pos.astype(str).str[2] == 'D'))]

dates = pd.date_range('2024-10-01', '2025-06-01')
games = get_schedule(dates)

2024-09-23 15:59:29.710 - ERROR - query.py - yfpy.query:291 - No data found when attempting extraction from fields: ['league', 'players']


In [349]:
ycols = ['g','a','pim','ppp','sog', 'fow', 'hit','blk', 'w','sv','so']
opp_exp = pd.DataFrame(s_exp).T[ycols]
exp = df[ycols].copy()
exp['gaa'] = df['gaa']
exp = exp.set_index(df.player)
exp = exp / 82
game_df = pd.DataFrame(games)
game_df = pd.concat([
    game_df.join(players.set_index('team')['name'], on='home'),
    game_df.join(players.set_index('team')['name'], on='away')
])
game_df = game_df.merge(exp, left_on='name', right_index=True)

In [350]:
with open('taken.txt', 'r') as f:
    taken = [s.strip() for s in f.readlines()]
with open('team.txt', 'r') as f:
    selected_team = [s.strip() for s in f.readlines()]


while len(selected_team) < 17:
    week_ind_preds = get_week_games(dates, game_df, players, selected_team)
    week_preds = week_ind_preds.groupby('name')[ycols].sum()

    current_exp = week_preds[~week_preds.index.isin(taken+selected_team)].copy()
    
    exp1 = get_added_vals(current_exp, opp_exp, selected_team)
    exp = exp1.sum(1)
    selected_team.append(exp.idxmax())
    print('TEAM\n', selected_team)

TEAM
 ['Mikko Rantanen', 'Steven Stamkos', 'Elias Pettersson', 'Nick Suzuki', 'Tim Stutzle', 'Mathew Barzal', 'John Tavares', 'Matt Boldy', 'Morgan Rielly', 'Brandon Montour', 'Mark Scheifele', 'John Carlson', 'Vince Dunn', 'Ukko-Pekka Luukkonen', 'Ilya Samsonov', 'Connor Ingram', 'Frank Vatrano']


In [325]:
players.set_index('name').loc[selected_team].pos

name
Mikko Rantanen           [RW]
Steven Stamkos        [C, LW]
Elias Pettersson      [C, LW]
Nick Suzuki               [C]
Tim Stutzle           [C, LW]
Mathew Barzal         [C, RW]
John Tavares              [C]
Matt Boldy           [LW, RW]
Morgan Rielly             [D]
Brandon Montour           [D]
Mark Scheifele            [C]
John Carlson              [D]
Dylan Guenther           [RW]
Frank Vatrano         [C, LW]
Vince Dunn                [D]
Seth Jones                [D]
Mikhail Sergachev         [D]
Name: pos, dtype: object